## A simple model for demand and supply of publicly-provided services in a city

In [ ]:
from enum import Enum
import os.path

import numpy as np
import pandas as pd
import geopandas as gpd
import geopy, geopy.distance
import shapely
from sklearn import gaussian_process

from matplotlib import pyplot as plt 
import seaborn as sns
plt.rcParams['figure.figsize']= (20,14)

In [ ]:
## TODO: find way to put this into some global settings
import os
import sys
nb_dir = os.path.dirname(os.getcwd())
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from references import common_cfg

In [ ]:
from src.models.city_items import AgeGroup, ServiceArea, ServiceType, SummaryNorm # enum classes for the model

In [ ]:
from src.models.services_supply import ServiceUnit, ServiceEvaluator, UnitFactory, SchoolFactory, LibraryFactory 
from src.models.services_supply import get_random_pos #TODO refactor this fun into common utils
from src.models.process_tools import MappedPositionsFrame, GridMaker #TODO refactor this fun into common utils

In [ ]:
quicktest = [ServiceUnit(ServiceType.PoliceStation, 'Duomo', ageDiffusionIn=None), 
        ServiceUnit(ServiceType.PoliceStation, 'Ripamonti', 
                    position=geopy.Point(45.43, 9.201), ageDiffusionIn=None)]

ServiceEvaluator(quicktest).evaluate_services_at(get_random_pos(4))
del quicktest

In [ ]:
## Load scuole
scuoleFile =  '../data/processed/Milano_datiScuole.csv'
schoolLoader = UnitFactory.createLoader(ServiceType.School, scuoleFile)

# Initialise with a default lengthscale of 0.5 km
schoolUnits = schoolLoader.load(meanRadius=0.5)

In [ ]:
## Load scuole
bibliotecheFile =  '../data/processed/Milano_biblioteche.csv'
bibliotecheLoader = UnitFactory.createLoader(ServiceType.Library, bibliotecheFile)

# Initialise with a default lengthscale of 0.5 km
libraryUnits = bibliotecheLoader.load(meanRadius=0.5)

In [ ]:
testgridmaking = GridMaker({'quartieri':'../data/raw/Milano_specific/Milano_quartieri.geojson'}, gridStep=.5)

In [ ]:
testgridmaking.mappedPositions

In [ ]:
class ServiceValues(dict):
    '''A class to store, make available for aggregation and easily export estimated service values'''
    
    def __init__(self, mappedPositions):
        assert isinstance(mappedPositions, MappedPositionsFrame), 'Expected MappedPositionsFrame'
        self.mappedPositions = mappedPositions
        
        # initialise for all service types
        super().__init__(
            {service: pd.DataFrame(np.zeros([mappedPositions.shape[0], len(AgeGroup.all())]),  
                                 index=[tuple(t) for t in mappedPositions.Positions], columns=AgeGroup.all()) 
                 for service in ServiceType})
    
    @property    
    def positions(self):
        return list(self.mappedPositions.Positions.values)

In [ ]:
class ServiceEvaluator:
    '''A class to evaluate a given list of service units'''
    
    def __init__(self, unitList, outputServicesIn=[t for t in ServiceType]):
        assert isinstance(unitList, list), 'List expected, got %s' % type(unitList)
        assert all([isinstance(t, ServiceUnit) for t in unitList]), 'ServiceUnits expected in list'
        self.units = unitList
        self.outputServices = outputServicesIn
    
    def evaluate_services_at(self, mappedPositions):
        assert isinstance(mappedPositions, MappedPositionsFrame), 'Expected MappedPositionsFrame'
        # set all age groups as output default
        outputAgeGroups = AgeGroup.all()
        # initialise output with dedicated class
        valuesStore = ServiceValues(mappedPositions)
        
        # loop over different services
        for thisServType in self.outputServices:
            serviceUnits = [u for u in self.units if u.service == thisServType]
            if not serviceUnits:
                continue
            else:
                for thisAgeGroup in outputAgeGroups:
                    unitValues = np.stack(list(map(
                        lambda x: x.evaluate(
                            valuesStore.positions, thisAgeGroup), serviceUnits)), axis=-1)
                    # aggregate unit contributions according to the service type norm
                    valuesStore[thisServType][thisAgeGroup] = thisServType.aggregate_units(unitValues)
        return valuesStore 

In [ ]:
class ValuesAggregator

In [ ]:
## Aggregation
def aggregate_values(self):
        
        assert self.serviceValues, 'Empty values on grid found, compute first please!'
        
        quartieri = self.subdivisions[self.quartiereKey][self.IDquartiereCol].values
        
        # initialize output
        averagedValues = {service: pd.DataFrame(np.zeros([len(quartieri), len(AgeGroup.all())]), 
                                 index=quartieri, columns=AgeGroup.all()) 
                 for service in self.serviceValues.keys()}
      
        leftOutZones = []
        for quartiere in quartieri:
            bThisQuartiere = (self.IDquartiere == quartiere)
            # loop over service types
            if not bThisQuartiere.any():
                print('Quartiere %s has no points in grid within it' % quartiere)
                leftOutZones.append(quartiere)
                continue
                
            for servType in self.serviceValues.keys():
                for ageGroup, valuesSeries in self.serviceValues[servType].items(): 
                    quartiereValues = [valuesSeries[tuple(p)] for p in self.plotPoints[bThisQuartiere]]
                    averagedValues[servType].loc[quartiere, ageGroup] = np.mean(quartiereValues)
                    
        return averagedValues, leftOutZones

In [ ]:
# Prepare service level aggregation by quartiere
schoolAggr = UnitAggregator(schoolUnits, 
                {'quartieri':'../data/raw/Milano_specific/Milano_quartieri.geojson'},
                IDQuartiereField = common_cfg.IdQuartiereColName,
                gridStep=.5)

In [ ]:
# Prepare service level aggregation by quartiere
libAggr = UnitAggregator(libraryUnits, 
                {'quartieri':'../data/raw/Milano_specific/Milano_quartieri.geojson'},
                IDQuartiereField = common_cfg.IdQuartiereColName,
                gridStep=.5)

In [ ]:
## Plot tools
from descartes import PolygonPatch

class unitPlotter:
    '''
    A class that plots various types of output from a UnitAggregator
    '''
    def __init__(self, unitAggregatorIn):
        assert isinstance(unitAggregatorIn, UnitAggregator), 'UnitAggregator class expected'
        self.ua = unitAggregatorIn
        
    def plot_locations(self):
        '''
        Plots the loaded ServiceUnits according to their locations and rescale them by the relative sizes
        '''
        plotScales = self.ua.scale/np.mean(self.ua.scale)
        plt.figure()
        plt.scatter(self.ua.longitude, self.ua.latitude, s=plotScales)
        plt.axis('equal')
        plt.show()
        return None
        
    def plot_grid_and_boundary(self):
        fig = plt.figure()
        ax = fig.gca()
        plt.scatter(self.ua.xPlot[self.ua.bInPerimeter], self.ua.yPlot[self.ua.bInPerimeter], cmap='k', s=1)
        ax.add_patch(PolygonPatch(self.ua.cityBoundary, fc='None',zorder=5))
        plt.axis('equal')
        plt.show()
        return None
        
    def plot_service_levels(self, servType):
        '''
        Plots a contour graph of the results for each ageGroup.
        '''
        assert isinstance(servType, ServiceType), 'ServiceType expected in input'
        
        for ageGroup, valuesSeries in self.ua.serviceValues[servType].items():
            valuesArray = np.vectorize(lambda p: valuesSeries[tuple(p)])(self.ua.plotPoints)
            if np.count_nonzero(valuesArray) > 0:
                plt.figure()
                plt.title(ageGroup)
                CS = plt.contourf(self.ua.xPlot, self.ua.yPlot, valuesArray, 40)
                cbar = plt.colorbar(CS)
                cbar.ax.set_ylabel('Service level')
                plt.show()
            
        return None

In [ ]:
plotterNew= unitPlotter(libAggr)
plotterNew.plot_grid_and_boundary()
#plotterNew.plot_service_levels(ServiceType.School)

In [ ]:
# Run evaluation
libAggr.compute_service_levels(ServiceType.Library)

In [ ]:
# Save and display aggregate values for zones
out, _ = schoolAggr.aggregate_values
finalSchoolData = out[ServiceType.Library].rename_axis(
    schoolAggr.IDquartiereCol)
finalSchoolData.to_csv('../data/output/Milano_librarySample.csv', encoding='utf-8')
finalSchoolData

In [ ]:
outtest = pd.read_csv('../data/output/Milano_librarySample.csv',
                     index_col=common_cfg.IdQuartiereColName)
outtest

In [ ]:
plotterNew.plot_service_levels(ServiceType.Library)